In [1]:
from openfermion.chem import MolecularData
from openfermionpsi4 import run_psi4
from openfermion.transforms import get_fermion_operator ,jordan_wigner
from ucc_operator import single_excitation_fix_ind,double_excitation_fix_ind,basis_trans,basis_backtrans
from ucc_circuit import gen_uccsd_anstaz,resolve_uccsd_ansatz
import numpy as np

# Set molecule parameters.
basis = '6-31g'
multiplicity = 1


# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Generate molecule at different bond lengths.

bond_length = 1.0

geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
molecule = MolecularData(
    geometry, basis, multiplicity,
    description=str(round(bond_length, 2)))

# Run Psi4.
molecule = run_psi4(molecule,
                    run_scf=run_scf,
                    run_mp2=run_mp2,
                    run_cisd=run_cisd,
                    run_ccsd=run_ccsd,
                    run_fci=run_fci)
print(molecule.hf_energy)

-1.0948079627740088


In [2]:
## hamiltontian
h_mol=molecule.get_molecular_hamiltonian()
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

In [3]:
## build circuit 
n_qubits=molecule.n_qubits
n_electron=molecule.n_electrons
n_orbitals=molecule.n_orbitals
n_occ=n_electron
n_vir=n_qubits-n_occ
print(f"HF sto-3g have {n_qubits} qubits")
print(f"HF sto-3g have {n_electron} electrons")
print(f"HF sto-3g have {n_orbitals} orbitals")
import cirq
my_qubits=[cirq.GridQubit(i,0) for i in range(n_qubits)]
ansatz=resolve_uccsd_ansatz(my_qubits,n_occ,n_vir)

# ## initial occupation state into 1 ,
# ## thus the state is \phi_0\alpha \phi_0 \beta \phi_1\alpha\phi_1\beta \cdots \phi_n \alpha \phi_n\beta
# def initial_circuit(n_electron,qubits):
#     for i in range(n_electron):
#         yield cirq.X(qubits[i]) 
   
import sympy
# c=cirq.Circuit()
# c.append(initial_circuit(n_electron,my_qubits))
n_single_excitation=int(n_occ*n_vir)
n_double_excitation=int(((n_occ-1)*n_occ)/2 *((n_vir-1)*n_vir)/2)
single_param=sympy.symbols(f"ts_0:{n_single_excitation}")
double_param=sympy.symbols(f"td_0:{n_double_excitation}")
# params=(single_param,double_param)
# ansatz=gen_uccsd_anstaz(n_occ,n_qubits,c,my_qubits,params)

HF sto-3g have 8 qubits
HF sto-3g have 2 electrons
HF sto-3g have 4 orbitals


In [4]:
## energy observable
from energy import get_energy_observable
energy_observable=get_energy_observable(jw_hamiltonian=jw_hamiltonian,qubits=my_qubits)

In [5]:
from vqe import energy_objective_vqe,optimize_amplitude

2025-01-13 09:41:28.421158: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-13 09:41:28.445211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-13 09:41:28.445242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-13 09:41:28.446122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 09:41:28.450466: I tensorflow/core/platform/cpu_feature_guar

In [6]:
initial_amplitudes=np.zeros(len(single_param)+len(double_param))
result=optimize_amplitude(energy_objective_vqe,initial_amplitudes,ts=single_param,td=double_param,
                   anstaz=ansatz,h_mol=energy_observable)

         Current function value: -1.126460
         Iterations: 9
         Function evaluations: 100
         Gradient evaluations: 89


/home/sydong/miniconda3/envs/openfermion/lib/python3.10/site-packages/scipy/optimize/_minimize.py:726: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


In [7]:
result.values()

dict_values([-1.1264595985412598, array([-0.00226879, -0.00119591,  0.00166512,  0.00094986, -0.00322247,
        0.00357246,  0.0014267 ,  0.00154591,  0.0002346 ,  0.00440693,
        0.00130749,  0.00202274,  0.00238037,  0.002738  ,  0.0014267 ,
        0.00106907,  0.00249958,  0.00309563,  0.002738  ,  0.00345325,
        0.00011539,  0.00762558,  0.00285721,  0.0002346 ,  0.00762558,
       -0.00071907,  0.00726795], dtype=float32), array([[ 2.28932431e+00, -2.64289188e-01, -1.38968558e+00,
        -1.81412577e-01, -9.55863281e-01, -1.72887944e+00,
        -4.61655882e-01, -9.95800171e-01, -1.94896915e+00,
        -1.12002301e+00, -8.92034063e-03, -2.73076224e+00,
        -5.43709811e-01, -2.61271515e+00, -1.67787066e-01,
         2.26037434e-01, -5.96931961e-01, -1.46936736e+00,
        -1.17772317e+00, -5.91634017e-01,  5.84079773e-01,
        -1.57140275e+00, -2.97016171e-02, -1.06129964e+00,
         2.80096435e-01,  1.74020580e-01, -9.19631517e-01],
       [-2.64289188e-01,

In [8]:
molecule.fci_energy

array(-1.12677835)